<a href="https://colab.research.google.com/github/ch00226855/CMP414765Spring2021/blob/main/Week13_ImageClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 13
# Image Classification with Convolutional Neural Network (CNN)

**Reference:** [TensorFlow Tutorial on Convolutional Neural Networks](https://www.tensorflow.org/tutorials/images/cnn)

**Please enable GPU computing before proceed.**

### Ideas
- Dense layers may contain redudent connections
- Some information should be invariant to spacial translation
- The number of parameters can be reduced if certain weights share the same value.

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcS4LZdFg5QPbgDb-jvP-YT0N51eRkWg45uF0ybsB5k0Ubr0-gOC&usqp=CAU" width="600">

## 2D Convolution Layer
<img src="https://cdn-media-1.freecodecamp.org/images/Gjxh-aApWTzIRI1UNmGnNLrk8OKsQaf2tlDu" width="600">

**2D smoothing with Gaussian kernel**

<img src="https://www.cs.umd.edu/class/fall2016/cmsc426/matlab/filters/html/filters_tutorial_03.png" width="600">

<img src="https://www.mathworks.com/help/examples/stats/win64/ComputeTheMultivariateNormalPdfExample_01.png" width="400">

**Edge detection**

<img src="https://aishack.in/static/img/tut/conv-line-detection-horizontal-result.jpg" width="400">

<img src="https://www.researchgate.net/profile/Ching-Wei_Wang/publication/221472523/figure/fig5/AS:305540338077700@1449857901164/Convolution-filter-for-simple-edge-detect.png" width="200">

## LeNet5 on MNIST

Yann LeCun, Leon Bottou, Yosuha Bengio and Patrick Haffner proposed a neural network architecture for handwritten and machine-printed character recognition in 1990’s which they called LeNet-5. It is one of the early example of a convolutional neural network

<img src="https://miro.medium.com/max/4308/1*1TI1aGBZ4dybR6__DI9dzA.png" width="800">

## Max-Pooling Layer
<img src="https://computersciencewiki.org/images/8/8a/MaxpoolSample2.png" width="500">

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
# import tensorflow.keras as K
print(tf.__version__)

In [ ]:
# Load and prepare the MNIST dataset.
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Convert the data from integers to floating-point numbers
x_train, x_test = x_train / 255.0, x_test / 255.0

print(x_train.shape, x_test.shape)

In [ ]:
model_cnn = tf.keras.models.Sequential()

model_cnn.add(tf.keras.layers.Conv2D(filters=6,
                                 kernel_size=(3, 3),
                                 activation='relu',
                                 input_shape=(28, 28, 1)))

model_cnn.add(tf.keras.layers.Conv2D(filters=16,
                                 kernel_size=(3, 3),
                                 activation='relu'))

model_cnn.add(tf.keras.layers.MaxPooling2D())

model_cnn.add(tf.keras.layers.Flatten())

model_cnn.add(tf.keras.layers.Dense(units=120,
                       activation='relu'))
model_cnn.add(tf.keras.layers.Dense(units=84,
                       activation='relu'))
model_cnn.add(tf.keras.layers.Dense(units=10,
                       activation='softmax'))

In [ ]:
model_cnn.summary()

In [ ]:
model_cnn.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model_cnn.fit(x_train.reshape(list(x_train.shape) + [1]), y_train, epochs=10)

In [ ]:
model_cnn.evaluate(x_test.reshape(list(x_test.shape) + [1]), y_test)

## Dropout and Model Regularization
For a complicated model like deep neural networks, a major concern on its performance is model overfitting:

![underfitting and overfitting](https://cdn-images-1.medium.com/max/1200/1*cdvfzvpkJkUudDEryFtCnA.png)

In plain words, overfitting happens when the model is **memorizing** the training data, and become poorly at **generalizing** what they've learned to unseen data. Think about a student who memorized the entire machine learning textbook. He may appear quite knowledgable in machine learning when asked things directly from the book, but there is no way he can perform a machine project on a dataset not mentioned in the book.

### How to dentify model overfitting?
- Visualize the model (decision boundary, regression curves, etc.)
- Observe the trends in training loss and the testing loss

![](https://cdn-images-1.medium.com/max/1600/1*vuZxFMi5fODz2OEcpG-S1g.png)

### How to prevent model overfitting?
1. Start with a simple model

![](https://image.slidesharecdn.com/lawsofwebdesign-091104020153-phpapp01/95/laws-of-web-development-11-728.jpg?cb=1257384621)
2. Add penalty to complicated models
    - L1 Regularizor
    - L2 Regularizor
    - Elastic Net

3. (For Neural Networks) Dropout layers: remove weights to the next layer

![](https://cdn-images-1.medium.com/max/1800/1*iWQzxhVlvadk6VAJjsgXgg.png)


# Image Classification with CIFAR-10 Dataset
[CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) is a widely used benchmark dataset for image classifiers. The dataset consists of 10 classes of color images of size $32\times 32$. Let's build a neural network with **convolutional layers** to classify the images.

### Download the dataset
- Use `request` to download the tar file from [https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz)
- Use `tarfile` to extract files
- Use `pickle` to load the data

In [ ]:
import requests

filename = "cifar-10-python.tar.gz"
if not os.path.isfile(filename):
    print("Downloading CIFAR10 dataset...")
    url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"
    file = requests.get(url)

    print("Writing to file", filename, "...")
    with open(filename, "wb") as f:
        f.write(file.content)

In [ ]:
import tarfile
datapath = "cifar-10-batches-py/" 
if not os.path.isdir(datapath):
    print("Extracting files...")
    tar = tarfile.open(filename)
    tar.extractall()
    print("Files extracted.")
    tar.close()
os.listdir(datapath)

In [ ]:
# load one batch
import pickle
with open(datapath + "data_batch_1", "rb") as f:
    batch = pickle.load(f, encoding="latin1")
    features = batch['data'].reshape([len(batch['data']), 3, 32, 32]).transpose(0, 2, 3, 1)
    labels = batch['labels']
print("feature size:", features.shape)
print("label size:", len(labels))

The label data is just a list of 10000 numbers in the range 0-9, which corresponds to each of the 10 classes in CIFAR-10. 

* **airplane**
* **automobile**
* **bird**
* **cat**
* **deer**
* **dog**
* **frog**
* **horse**
* **ship**
* **truck**

In [ ]:
# Show a sample image
sample_id = 4321
plt.imshow(features[sample_id])
label_names = ['airplane', 'automobile', 'bird',
            'cat', 'deer', 'dog', 'frog',
            'horse', 'ship', 'truck']
plt.xlabel(label_names[labels[sample_id]])

In [ ]:
# Load all images from batch 1-5
train_features = np.empty([0, 32, 32, 3], dtype=np.uint8)
train_labels = np.empty([0])
for k in range(1, 6):
    with open(datapath + "data_batch_" + str(k), "rb") as f:
        batch = pickle.load(f, encoding="latin1")
        features = batch["data"].reshape([len(batch['data']), 3, 32, 32]).transpose(0, 2, 3, 1)
        labels=batch['labels']
        print("features shape:", features.shape)
        print("labels shape:", len(labels))
        train_features = np.append(train_features, features, axis=0)
        train_labels = np.append(train_labels, labels, axis=0)
print("train_features shape:", train_features.shape)
print("train_labels shape:", train_labels.shape)
        

## Build CNN model
### Create Convolutional Model

The entire model consists of 14 layers in total. In addition to layers below lists what techniques are applied to build the model.

1. Convolution with 32 different filters in size of (3x3)
    - ReLU activation function 
2. Convolution with 32 different filters in size of (3x3)
    - ReLU activation function 
    - Max Pooling by 2
    - Dropout 
3. Convolution with 64 different filters in size of (3x3)
    - ReLU activation function 
4. Convolution with 64 different filters in size of (3x3)
    - ReLU activation function 
    - Max Pooling by 2
    - Dropout 
5. Flattening the 3-D output of the last convolutional operations.
6. Fully Connected Layer with 512 units
  - Dropout 
7. Fully Connected Layer with 10 units (number of image classes)

the image below decribes how the conceptual convolving operation differs from the tensorflow implementation when you use [Channel x Width x Height] tensor format. 

<img src="https://adeshpande3.github.io/assets/Cover.png" alt="Drawing" style="width: 1000px;"/>

In [ ]:
batch_size = 32 # How many images to load at a time
num_classes = 10 
epochs = 10
num_predictions = 20

In [ ]:
# Build CNN model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same",
                              input_shape=features[0].shape, # (32, 32, 3)
                              activation=tf.nn.relu))
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding="same",
                              activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same",
                              activation=tf.nn.relu))
model.add(tf.keras.layers.Conv2D(64, (3, 3), padding="same",
                              activation=tf.nn.relu))
model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.25))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation=tf.nn.relu))
model.add(tf.keras.layers.Dropout(0.5))
# model.add(tf.keras.layers.Dense(num_classes, activation=tf.nn.softmax))
model.add(tf.keras.layers.Dense(num_classes))

In [ ]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=["accuracy"])

In [ ]:
# Normalize data
def normalize(x):
    """
        argument
            - x: input image data in numpy array [32, 32, 3]
        return
            - normalized x 
    """
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x

In [ ]:
train_features_scaled = normalize(train_features)

In [ ]:
history =  model.fit(train_features_scaled, train_labels, epochs=10, batch_size=batch_size)

In [ ]:
# Evaluate the model
plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy') # allocate validation data to get val_accuracy
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
# Load test images
with open(datapath + "test_batch", "rb") as f:
    batch = pickle.load(f, encoding="latin1")
    test_features = batch['data'].reshape([len(batch['data']), 3, 32, 32]).transpose(0, 2, 3, 1)
    test_labels = batch['labels']

In [ ]:
# Normalize test features
test_features_scaled = normalize(test_features)

In [ ]:
test_labels = np.array(test_labels)
test_labels.shape

In [ ]:
test_loss, test_acc = model.evaluate(test_features_scaled,  test_labels, verbose=2)
print(test_acc)